In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hi\\Desktop\\Chest-Cancer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hi\\Desktop\\Chest-Cancer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
# dataclass create a class entity without self
@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir:Path
  source_URL:str
  local_data_file:Path
  unzip_dir:Path